In [34]:
# Import libraries that are required to run your project
# You are allowed to add more libraries as you need

import pandas as pd
import numpy as np
from scipy.stats import spearmanr

## Work Package 1.1 - Modeling Choices & Data Pre-processing

In [35]:
# TODO: 
# Load your feature (bed and/or bigwig and/or fasta) and target files (tsv) here.
# Decide which features to use for training. Feel free to process them however you need.

# NOTE: 
# bed and bigwig files contain signals of all chromosomes (including sex chromosomes).
# Training and validation split based on chromosomes has been done for you. 
# However, you can resplit the data in any way you want.

#path_data = "/path/to/your/data/files"  # TODO
#path_test = "/path/to/test/info/file"   # X3_test_info.tsv ; TODO
#test_genes = pd.read_csv(path_test, sep='\t')
# ---------------------------INSERT CODE HERE---------------------------


    
path = "./Data/CAGE-train/"

X1_train = pd.read_csv(path + "X1_train_info.tsv", sep= '\t')
X1_train_y = pd.read_csv(path + "X1_train_y.tsv", sep= '\t')

X1_val = pd.read_csv(path + "X1_val_info.tsv", sep= '\t')
X1_val_y = pd.read_csv(path + "X1_val_y.tsv", sep= '\t')

X2_train = pd.read_csv(path + "X2_train_info.tsv", sep= '\t')
X2_train_y = pd.read_csv(path + "X2_train_y.tsv", sep= '\t')

X2_val = pd.read_csv(path + "X2_val_info.tsv", sep= '\t')
X2_val_y = pd.read_csv(path + "X2_val_y.tsv", sep= '\t')

X3_test = pd.read_csv(path + "X3_test_info.tsv", sep= '\t')



# get all .bed datasets in a dictionary
df_DNase_histones = {'DNase':[], 'H3K4me1':[], 'H3K4me3':[], 'H3K27ac':[], 'H3K27me3':[], 'H3K36me3':[]}
for name, df_cell_list in df_DNase_histones.items():
    path = './Data/'+name+'-bed/'
    for i in range(1,4):
        X = pd.read_csv(path + 'X' + str(i) + ".bed", sep= '\t')
        X.columns = ['chr', 'start','end', 'name','display_score','strand','signal_value',
                                  'pValue','qValue','peak']
        df_cell_list.append(X)
        
    
# Function to get DNase and histones data and the maximum values of peaks for each gene
def window_peaks_data(window_data_df, df_DNase_histones, cell_line_num):
    
    maxSignalVal_list = []
    
    for name, df_list in df_DNase_histones.items():
    
        df = df_list[cell_line_num-1] # cell line's DNase or histone data
        
        if name == 'H3K36me3':
                peak_data = df.loc[(df['chr'] == window_data_df['chr']) & 
                                      (df['start'] >= window_data_df['gene_start']) & 
                                      (df['end'] <= window_data_df['gene_end'])]
        else:
            peak_data = df.loc[(df['chr'] == window_data_df['chr']) & 
                                  (df['start'] >= window_data_df['window_start']) & 
                                  (df['end'] <= window_data_df['window_end'])]
        if len(peak_data) == 0:
            maxSignalVal = 0
        else:
            # get row from .bed file with max signal value
            peak_data_max = peak_data.loc[peak_data['signal_value'].idxmax()]
            maxSignalVal = peak_data_max['signal_value']

        maxSignalVal_list.append(maxSignalVal)

    return maxSignalVal_list


# Function to get maximum value of peaks of DNase and histones for each gene
def getMaxSignalData(X_train, df_DNase_histones, window_size, cell_line_num):

    window_data_df = pd.DataFrame(X_train[['gene_name', 'chr', 'gene_start', 'gene_end', 'TSS_start']])
    window_data_df['window_start'] = X_train['TSS_start']-window_size
    window_data_df['window_end'] = X_train['TSS_start']+window_size


    maxSignalVal_list = window_data_df.apply(window_peaks_data, args=[df_DNase_histones, cell_line_num], axis=1)

    # process maxSignalVal_list to see data in dataframe
    maxSignalVal_df = pd.DataFrame(maxSignalVal_list)
    maxSignalVal_df = maxSignalVal_df[0].apply(pd.Series)
    maxSignalVal_df.columns = ['DNase','H3K4me1','H3K4me3','H3K27ac','H3K27me3','H3K36me3']
    
    return maxSignalVal_df



# Generate datasets to be used for training model

# window_size = Num of kb region to scan around TSS
window_size = 3000

# X1 Train dataset
maxSignalVal_X1_train = getMaxSignalData(X1_train, df_DNase_histones, window_size, cell_line_num=1)
x1_train = maxSignalVal_X1_train.values
y1_train = X1_train_y['gex'].values

# X1 Validation dataset
maxSignalVal_X1_val = getMaxSignalData(X1_val, df_DNase_histones, window_size, cell_line_num=1)
x1_val = maxSignalVal_X1_val.values
y1_val = X1_val_y['gex'].values

# X2 Train dataset
maxSignalVal_X2_train = getMaxSignalData(X2_train, df_DNase_histones, window_size, cell_line_num=2)
x2_train = maxSignalVal_X2_train.values
y2_train = X2_train_y['gex'].values

# X2 Validation dataset
maxSignalVal_X2_val = getMaxSignalData(X2_val, df_DNase_histones, window_size, cell_line_num=2)
x2_val = maxSignalVal_X2_val.values
y2_val = X2_val_y['gex'].values

# X3 Test dataset
maxSignalVal_X3_test = getMaxSignalData(X3_test, df_DNase_histones, window_size, cell_line_num=3)
x3_test = maxSignalVal_X3_test.values

# X1 + X2 cell lines
x_train = np.concatenate((x1_train, x2_train), axis=0)
y_train = np.concatenate((y1_train, y2_train), axis=0)
x_val = np.concatenate((x1_val, x2_val), axis=0)
y_val = np.concatenate((y1_val, y2_val), axis=0)


# ---------------------------------------------------------------------- 

## Work Package 1.2 - Model Building

In [36]:
# TODO: 
# Select the best model to predict gene expression from the obtained features in WP 1.1.

# ---------------------------INSERT CODE HERE---------------------------


from sklearn.svm import SVR


# X1 cell line
reg1 = SVR().fit(x1_train, y1_train)
y1_pred = reg1.predict(x1_val)
y1_pred = np.where(y1_pred<0, 0, y1_pred)
print('X1 Val: ', spearmanr(y1_val, y1_pred))

# X2 cell line
reg2 = SVR().fit(x2_train, y2_train)
y2_pred = reg2.predict(x2_val)
y2_pred = np.where(y2_pred<0, 0, y2_pred)
print('X2 Val: ', spearmanr(y2_val, y2_pred))


# X1 + X2 cell lines
reg3 = SVR().fit(x_train, y_train)
y_pred = reg3.predict(x_val)
y_pred = np.where(y_pred<0, 0, y_pred)
print('X1 + X2 Val: ', spearmanr(y_val, y_pred))



# ----------------------------------------------------------------------


X1 Val:  SpearmanrResult(correlation=0.6754959215224289, pvalue=3.1116598955001695e-263)
X2 Val:  SpearmanrResult(correlation=0.629625272999413, pvalue=1.7943344509695703e-218)
X1 + X2 Val:  SpearmanrResult(correlation=0.6807620045429184, pvalue=0.0)


## Work Package 1.3 - Prediction on Test Data (Evaluation Metric)

In [40]:
# TODO:
# Using the model trained in WP 1.2, make predictions on the test data (chr 1 of cell line X3).
# Store predictions in a variable called "pred" which is a numpy array.

pred = None
# ---------------------------INSERT CODE HERE---------------------------


pred = reg3.predict(x3_test)
pred = np.where(pred<0, 0, pred)


# ----------------------------------------------------------------------

# Check if "pred" meets the specified constrains
assert isinstance(pred, np.ndarray), 'Prediction array must be a numpy array'
assert np.issubdtype(pred.dtype, np.number), 'Prediction array must be numeric'
assert pred.shape[0] == len(X3_test), 'Each gene should have a unique predicted expression'

#### Store Predictions in the Required Format

In [41]:
# Store predictions in a ZIP. 
# Upload this zip on the project website under "Your submission".
# Zip this notebook along with the conda environment (and README, optional) and upload this under "Your code".

save_dir = './output'  # TODO
file_name = 'gex_predicted.csv'         # PLEASE DO NOT CHANGE THIS
zip_name = "Agrawal_Nidhi_Project1.zip" # TODO
save_path = f'{save_dir}/{zip_name}'
compression_options = dict(method="zip", archive_name=file_name)

X3_test['gex_predicted'] = pred.tolist()
X3_test[['gene_name', 'gex_predicted']].to_csv(save_path, compression=compression_options)
